# NPS adjustments

Before trainning the model, there are a few caveats about the NPS and the touchpoints' data. Particularly, there are 3 adjustments we have to consider:
1) Monthly statistical adjustment (weights): Each month, the NPS scores per cabin/haul are adjusted to fit a true sample of the global distribution of cabin/haul users. That is, for the NPS per cabin haul to be representative, it is first adjusted so that the sample of users from which it is computed is representative as well. That way, if by chance a month has only one passenger in, for example, Bussines-Long Haul, the associated NPS considers that that particular sample is not representtive of the normal distribution of Business Long Haul users. Unfortunatelly, this values are computed every month, and we wont have access to them at prediction time. 
2) Change of survey: On early december 2019 and on late december 2023 there have been changes on the survey handled to the clients that have considerelly affected the NPS score and the satisfaction per touchpoint. A fixed value (per cabin/haul and globally) is used to adjust 2019 and 2022 (plus december 2019). Those values are computed externally.
3) C/H ponderation: On the global NPS computation, each cabin haul NPS is adjusted by a factor based on the percentaje of users that will fly on that particular cabine/haul. These values are fixed and at the moment of prediction we will have access to them.

## Installs and imports

In [2]:
!pip install plotly boto3==1.19.12 s3fs pandas==2.1.2 


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# General
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import os
import numpy as np
import xlsxwriter
import datetime
import boto3


# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

# Plots
import matplotlib.pyplot as plt
import seaborn as sns

#Warnings
import warnings
warnings.filterwarnings("ignore")

## Read files

In [4]:
insert_date_ci="2023-11-21"
nps_path = f's3://iberia-data-lake/customer/nps_surveys/export_historic/insert_date_ci={insert_date_ci}/'
iberia_kpis_path = 's3://iberia-data-lake/customer/one_shot/iberia_kpis_monthly_fixed/'

if not os.path.exists(f'outputs/{insert_date_ci}'):
    os.makedirs(f'outputs/{insert_date_ci}')

outputs_path= f'outputs/{insert_date_ci}/monthly_NPS_{insert_date_ci}_fixed.csv'

In [5]:
import boto3
import pandas as pd

def read_csv_files_from_s3(s3_path):
    # Initialize Boto3 client for S3
    s3_client = boto3.client('s3')

    # Extract bucket name and prefix from the S3 path
    bucket, prefix = s3_path.replace('s3://', '').split('/', 1)

    # Get a list of all keys (object paths) under the specified S3 path
    s3_resource = boto3.resource('s3')
    s3_keys = [item.key for item in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix)]

    # Generate a list of full S3 paths for all CSV files
    preprocess_paths = [f"s3://{bucket}/{key}" for key in s3_keys]

    # Read all CSV files into a list of DataFrames
    dfs = [pd.read_csv(s3_client.get_object(Bucket=bucket, Key=key)['Body']) for key in s3_keys]

    # Concatenate DataFrames into a single DataFrame
    df_concatenated = pd.concat(dfs, axis=0, ignore_index=True)

    return df_concatenated

In [6]:
# Readd files from paths
df_nps = read_csv_files_from_s3(nps_path)

In [7]:
df_nps[df_nps['monthly_weight'].notna()]['date_flight_local'].max()

'2023-10-31'

In [8]:
[col for col in df_nps.columns if 'ease_contact' in col]

['img_310_ease_contact_phone',
 'img_320_ease_contact_ibplus_mail',
 'img_330_ease_contact_social_net']

In [9]:
df_kpis = read_csv_files_from_s3(iberia_kpis_path)

In [10]:
df_kpis[(df_kpis['year']==2023) & (df_kpis['month']==9)]

,year,month,haul,cabin,load_factor,otp15_landing,otp15_takeoff
24,2023,9,SH,Business,73.102,84.182,87.551
104,2023,9,LH,Economy,86.796,83.828,88.696
181,2023,9,SH,Economy,83.168,84.182,87.551
223,2023,9,SH,Premium Economy,26.923,84.182,87.551
256,2023,9,LH,Premium Economy,88.910,83.828,88.696
317,2023,9,LH,Business,91.363,83.828,88.696


## 1) Monthly statistical adjustment.

In [11]:
condition_1 = (df_nps['operating_airline_code'].isin(['IB', 'YW']))
condition_2 = ((df_nps['invitegroup_ib'] != 3) | (df_nps['invitegroup_ib'].isnull()))
condition_3 = (df_nps['invitegroup'] == 2)

df_nps_tkt = df_nps.loc[condition_1 & (condition_2 & condition_3)]

In [12]:
df_nps

,respondent_id,sample_id,surveyed_flight_number,date_flight_local,scheduled_departure_time_local,scheduled_arrival_time_local,tier_level,language_code,aircraft_registration_number,seat_no,volume_of_bags,number_of_child_in_the_booking,number_of_infant_in_the_booking,number_of_people_in_the_booking,infinita_customers_identifer,flag_of_ib_singular_customers,country_code,list_of_options_for_booking_channel,list_of_options_for_checkin_channel,lounge_used_at_origin_airport,customer_journey_origin,customer_journey_destination,number_of_flights_in_journey,order_of_flight_in_journey,fleet_in_surveyed_flight,marketing_airline_code,date_of_flight_gmt,scheduled_departure_time_gmt,real_departure_time_local,real_departure_time_gmt,scheduled_arrival_time_gmt,real_arrival_time_local,real_arrival_time_gmt,segment,route,overall_haul,purser,invitegroup_ib,weight_category,weekly_weight,monthly_weight,pnr_show,ff_number,id_golden_record,ticket_num,started,time_spent_hrminsec,customer_email_show,origin_of_surveyed_flight,destination_of_surveyed_flight,operating_airline_code,cabin_in_surveyed_flight,haul,first_name_show,last_name_show,second_last_name_show,nps_category,nps_100,survey_type,invitegroup,group_age_survey,pun_100_punctuality,inm_200_issues_prior_checkin,inm_200_issues_prior_ticket_change,inm_200_issues_prior_schedule_change,inm_200_issues_prior_contact_center,inm_200_issues_prior_special_serv,inm_200_issues_prior_special_req,inm_200_issues_prior_avios,inm_200_issues_prior_voucher,inm_206_issues_checkin_long_queues,inm_206_issues_checkin_wrong_info,inm_206_issues_checkin_additional_fees,inm_206_issues_checkin_overbooking,inm_206_issues_checkin_downgrade,inm_206_issues_checkin_staff,inm_206_issues_checkin_social_distance,inm_206_issues_checkin_face_masks,inm_206_issues_checkin_documentation,inm_206_issues_checkin_other,inm_207_issues_lounge_denied,inm_207_issues_lounge_overcrowded,inm_207_issues_lounge_cleanliness,inm_207_issues_lounge_wifi,inm_207_issues_lounge_staff,inm_207_issues_lounge_food_drink,inm_207_issues_lounge_face_masks,inm_207_issues_lounge_other,inm_208_issues_security_leave_sth,inm_208_issues_security_long_queues,inm_208_issues_security_staff,inm_208_issues_security_social_distance,inm_208_issues_security_face_masks,inm_208_issues_security_other,inm_209_issues_boarding_unclear,inm_209_issues_boarding_gate_changed,inm_209_issues_boarding_lack_space,inm_209_issues_boarding_disorganised,inm_209_issues_boarding_staff,inm_209_issues_boarding_social_distance,inm_209_issues_boarding_face_masks,inm_209_issues_boarding_documentation,inm_209_issues_boarding_other,inm_220_issues_timing_cancelled,inm_220_issues_timing_dep_delay,inm_220_issues_timing_arr_delay,inm_220_issues_timing_missed,inm_230_issues_onboard_staff,inm_230_issues_onboard_ife,inm_230_issues_onboard_overcrowding,inm_230_issues_onboard_face_masks,inm_235_issues_onboard_comfort_damaged,inm_235_issues_onboard_comfort_space,inm_235_issues_onboard_comfort_temperature,inm_235_issues_onboard_comfort_cleanliness,inm_235_issues_onboard_comfort_washrooms,inm_235_issues_onboard_comfort_other,inm_236_issues_meal_availability,inm_236_issues_meal_portions,inm_236_issues_meal_quality,inm_236_issues_meal_special,inm_236_issues_meal_other,inm_240_issues_baggage_lost,inm_240_issues_baggage_delayed,inm_240_issues_baggage_demaged,inm_240_issues_baggage_staff,inm_240_issues_baggage_hand,inm_240_issues_baggage_other,inm_250_issues_arrival_slow,inm_250_issues_arrival_unclear,inm_250_issues_arrival_aditional_request,inm_250_issues_arrival_staff,inm_250_issues_arrival_immigration_queues,inm_250_issues_arrival_immigration_passport,inm_250_issues_arrival_immigration_other,inm_255_issues_connecting_missed,inm_255_issues_connecting_staff,inm_255_issues_connecting_baggage,inm_255_issues_connecting_other,bkg_100_booking,bkg_200_journey_preparation,inm_400_issues_response,pfl_100_checkin,pfl_200_security,pfl_300_lounge,pfl_500_boarding,ifl_100_cabin_crew,ifl_200_flight_crew_annoucements,ifl_300_cabin,

In [13]:
datetime_features = ['date_flight_local', 'scheduled_departure_time_local', 'scheduled_arrival_time_local', 'real_departure_time_local',
                     'real_arrival_time_local', 'started']
columns_to_cross_kpis=['cabin_in_surveyed_flight','haul']
columns_ext = ['tier_level', 'language_code', 'seat_no', 'volume_of_bags', 'number_of_child_in_the_booking', 'number_of_infant_in_the_booking',
              'number_of_people_in_the_booking', 'country_code', 'customer_journey_origin', 'customer_journey_destination', 'number_of_flights_in_journey',
              'order_of_flight_in_journey', 'marketing_airline_code', 'overall_haul', 'weight_category', 'ff_number', 'ticket_num', 'operating_airline_code',
               'nps_category', 'nps_100', 'group_age_survey', 'gender'] # invite_group

#'bkg_100_booking', 
touchpoints = ['bkg_200_journey_preparation', 'pfl_100_checkin', 'pfl_200_security', 'pfl_300_lounge',
               'pfl_500_boarding', 'ifl_300_cabin', 'ifl_200_flight_crew_annoucements', 'ifl_600_wifi', 'ifl_500_ife',
               'ifl_400_food_drink', 'ifl_100_cabin_crew', 'arr_100_arrivals', 'con_100_connections', 'pun_100_punctuality',
               'loy_200_loyalty_programme', 'inm_400_issues_response', 'img_310_ease_contact_phone']

# ,'img_320_ease_contact_ibplus_mail'
survey_fields = ['cla_600_wifi_t_f', 'tvl_journey_reason']

df_nps_tkt['date_flight_local'] = pd.to_datetime(df_nps_tkt['date_flight_local'], errors = 'ignore')


In [14]:
df_nps_tkt=df_nps_tkt[df_nps_tkt['date_flight_local'].dt.year >= 2019]

In [15]:
df_nps_tkt['date_flight_local'].min()

Timestamp('2019-01-01 00:00:00')

In [16]:
columns_to_select = datetime_features + columns_ext + touchpoints +columns_to_cross_kpis+['monthly_weight']

In [17]:
df_nps_tkt = df_nps_tkt[columns_to_select]
df_nps_tkt[['date_flight_local','nps_100','monthly_weight']]

,date_flight_local,nps_100,monthly_weight
1,2023-01-04,8,0.519
2,2023-01-29,10,0.414
3,2019-07-23,9,1.593
4,2019-09-03,6,0.729
6,2019-02-16,9,1.502
...,...,...,...
915849,2023-11-16,10,NaN
915850,2023-10-05,9,0.473
915851,2023-09-07,9,0.476
915852,2023-10-12,10,0.367


In [18]:
x=df_nps_tkt.copy()
pd.set_option('display.max_rows', 100)
x = x[(x['date_flight_local']=='2022-12-01') & (x['cabin_in_surveyed_flight'] == 'Economy') & (x['haul'] == 'MH') ]
display(x[(x['nps_100'] > 8) | (x['nps_100'] < 6)][['date_flight_local','cabin_in_surveyed_flight','haul','monthly_weight','nps_100']].head(200))



,date_flight_local,cabin_in_surveyed_flight,haul,monthly_weight,nps_100
14391,2022-12-01,Economy,MH,2.708,9
27782,2022-12-01,Economy,MH,2.708,10
54798,2022-12-01,Economy,MH,2.708,9
66610,2022-12-01,Economy,MH,2.708,9
66624,2022-12-01,Economy,MH,2.708,3
68278,2022-12-01,Economy,MH,2.708,5
68419,2022-12-01,Economy,MH,2.708,10
68450,2022-12-01,Economy,MH,2.708,5
90665,2022-12-01,Economy,MH,2.708,10
90863,2022-12-01,Economy,MH,2.708,10


#### If 4 cabin/haul

Before aggregating, I need to rewrite come categories in order to comapre with the dashboard:

In [19]:
df_nps_tkt['haul'] = df_nps_tkt['haul'].replace('MH', 'SH')
#df_nps_tkt['cabin_in_surveyed_flight'] = df_nps_tkt['cabin_in_surveyed_flight'].replace('Premium Economy', 'Economy')

#df_kpis['cabin']=df_kpis['cabin'].replace('Premium Economy','Premium Economy')
df_kpis['cabin']=df_kpis['cabin'].replace('Premium','Business')


In [20]:
df_nps_tkt[df_nps_tkt['monthly_weight'].isnull()][['date_flight_local','cabin_in_surveyed_flight','haul']]

,date_flight_local,cabin_in_surveyed_flight,haul
32769,2023-06-30,Economy,SH
33783,2023-07-31,Premium Economy,LH
33927,2023-07-31,Economy,SH
34934,2023-11-12,Premium Economy,LH
34937,2023-11-02,Economy,LH
...,...,...,...
915814,2023-11-05,Economy,SH
915823,2023-11-14,Economy,SH
915844,2023-11-10,Economy,LH
915849,2023-11-16,Economy,LH


In [21]:
df_nps_tkt = df_nps_tkt[df_nps_tkt['monthly_weight'].notnull()]

### Filter dataframes on the strike dates

In [22]:
df_nps_tkt = df_nps_tkt.query(
    '(date_flight_local <= "2023-03-07") | '
    '(date_flight_local >= "2023-03-14") & (date_flight_local <= "2023-03-23") | '
    '(date_flight_local >= "2023-03-26")'
)

In [23]:
df_nps_tkt

,date_flight_local,scheduled_departure_time_local,scheduled_arrival_time_local,real_departure_time_local,real_arrival_time_local,started,tier_level,language_code,seat_no,volume_of_bags,number_of_child_in_the_booking,number_of_infant_in_the_booking,number_of_people_in_the_booking,country_code,customer_journey_origin,customer_journey_destination,number_of_flights_in_journey,order_of_flight_in_journey,marketing_airline_code,overall_haul,weight_category,ff_number,ticket_num,operating_airline_code,nps_category,nps_100,group_age_survey,gender,bkg_200_journey_preparation,pfl_100_checkin,pfl_200_security,pfl_300_lounge,pfl_500_boarding,ifl_300_cabin,ifl_200_flight_crew_annoucements,ifl_600_wifi,ifl_500_ife,ifl_400_food_drink,ifl_100_cabin_crew,arr_100_arrivals,con_100_connections,pun_100_punctuality,loy_200_loyalty_programme,inm_400_issues_response,img_310_ease_contact_phone,cabin_in_surveyed_flight,haul,monthly_weight
1,2023-01-04,2023-01-04 13:35:00,2023-01-04 13:35:00,2023-01-04 13:55:00,2023-01-04 14:00:00,2023-01-07 15:30:09,NaN,EN,07F,TRUE,0.000,0,3,NaN,CDT,MAD,1.000,1.000,IB,SH,ECONOMY_SH+MH,NaN,NaN,YW,Passive,8,45-54,Female,8.000,9.000,4.000,NaN,9.000,9.000,9.000,NaN,NaN,7.000,9.000,9.000,NaN,7.000,NaN,NaN,NaN,Economy,SH,0.519
2,2023-01-29,2023-01-29 16:50:00,2023-01-29 18:30:00,2023-01-29 16:45:00,2023-01-29 18:07:00,2023-02-01 15:33:18,IB SILVER (OW RUBY),ES,01A,FALSE,0.000,0,1,NaN,LEU,MAD,1.000,1.000,IB,SH,BUSINESS_SH+MH,NaN,NaN,YW,Promoter,10,35-44,Male,9.000,10.000,9.000,NaN,9.000,8.000,NaN,NaN,NaN,9.000,10.000,10.000,NaN,10.000,6.000,NaN,NaN,Business,SH,0.414
3,2019-07-23,2019-07-23 23:55:00,2019-07-24 00:00:00,2019-07-23 00:16:00,2019-07-23 07:54:00,2019-07-26 07:08:24,IB GOLD (OW ZAPHIRE),ES,NaN,NaN,NaN,0,1,ES,MAD,BUE,1.000,1.000,IB,LH,ECONOMY_LH,13691753,NaN,IB,Promoter,9,45-54,M,8.000,8.000,8.000,8.000,8.000,9.000,9.000,6.000,8.000,7.000,9.000,8.000,NaN,8.000,9.000,NaN,8.000,Economy,LH,1.593
4,2019-09-03,2019-09-03 12:10:00,2019-09-03 19:50:00,2019-09-03 12:20:00,2019-09-03 19:41:00,2019-09-10 07:25:17,NaN,ES,NaN,NaN,NaN,0,2,NaN,BCN,BUE,1.000,1.000,IB,LH,ECONOMY_LH,NaN,NaN,IB,Detractor,6,65+,NaN,8.000,7.000,8.000,NaN,7.000,8.000,8.000,NaN,NaN,8.000,8.000,8.000,NaN,8.000,NaN,NaN,6.000,Economy,LH,0.729
6,2019-02-16,2019-02-16 19:50:00,2019-02-16 20:55:00,2019-02-16 19:53:00,2019-02-16 21:00:00,2019-02-19 06:05:20,IB CLASSIC,ES,NaN,NaN,NaN,0,1,ES,IBZ,BIO,2.000,2.000,IB,SH,ECONOMY_SH+MH,ANÓNIMO,NaN,IB,Promoter,9,26-34,F,7.000,6.000,NaN,NaN,7.000,8.000,8.000,NaN,NaN,NaN,NaN,8.000,7.000,10.000,8.000,NaN,5.000,Economy,SH,1.502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915847,2023-10-24,2023-10-24 17:50:00,2023-10-25 14:10:00,2023-10-24 17:56:00,2023-10-25 13:17:00,2023-10-27 15:51:27,IB GOLD (OW ZAPHIRE),ES,32C,False,0.000,0,1,NaN,SFO,MAD,1.000,1.000,AA,LH,ECONOMY_LH,NaN,NaN,IB,Promoter,10,45-54,Male,NaN,10.000,10.000,NaN,10.000,9.000,9.000,2.000,NaN,9.000,10.000,10.000,NaN,10.000,NaN,NaN,NaN,Economy,LH,0.339
915848,2023-10-05,2023-10-05 22:35:00,2023-10-06 13:35:00,2023-10-05 22:26:00,2023-10-06 13:10:00,2023-10-09 16:24:00,IB CLASSIC,ES,24H,True,0.000,0,1,NaN,MIA,MAD,2.000,2.000,IB,LH,ECONOMY_LH,53703864,751421457268.000,IB,Detractor,0,55-64,Male,0.000,9.000,9.000,NaN,8.000,8.000,10.000,NaN,NaN,9.000,10.000,10.000,NaN,10.000,NaN,0.000,NaN,Economy,LH,0.339
915850,2023-10-05,2023-10-05 21:25:00,2023-10-05 23:50:00,2023-10-05 21:22:00,2023-10-05 23:42:00,2023-10-09 09:42:25,IB CLASSIC,ES,31F,False,0.000,0,2,NaN,MAD,VCE,2.000,1.000,IB,MH,ECONOMY_SH+MH,44169209,751419913546.000,IB,Promoter,9,35-44,Male,NaN,10.000,8.000,NaN,9.000,9.000,8.000,NaN,NaN,9.000,9.000,9.000,NaN,10.000,10.000,NaN,NaN,Economy,SH,0.473
915851,2023-09-07,2023-09-07 15:45:00,2023-09-07 18:25:00,2023-09-07 16:02:00,2023-09-07 18:25:00,2023-09-11 18:35:34,IB CLASSIC,ES,09D,True,0.000,0,1,NaN,FCO,BIO,4.000,3

### Aggregate on time frequency

In [24]:
# Function to calculate NPS
def calculate_nps(promoters, detractors, passives):
    total_responses = promoters + detractors + passives
    nps = 100 * (promoters - detractors) / total_responses
    return nps

def calculate_aggregated_features_with_cross_kpis(df, variables, columns_to_cross_kpis, time_frequency='M', calculate_satisfaction=True):
    touchpoints = [var for var in variables if calculate_satisfaction]
    non_touchpoints = [var for var in variables if not calculate_satisfaction]

    # Combine the values of columns_to_cross_kpis into a new column 'cross_kpis'
    df['cross_kpis'] = df[columns_to_cross_kpis].apply(lambda x: '_'.join(x.values.astype(str)), axis=1)

    # Group the data by 'cross_kpis' and 'date_flight_local' using pd.Grouper on the entire DataFrame
    grouped_data = df.groupby([pd.Grouper(key='cross_kpis'), pd.Grouper(key='date_flight_local', freq=time_frequency)], group_keys=False)

    # Calculate the aggregated features for each group
    aggregated_features_data = pd.DataFrame()
    
    # Calculate the aggregated features for each group
    aggregated_features_data['NPS'] = grouped_data.apply(lambda x: calculate_nps(
        sum((x['nps_100'] == 9) | (x['nps_100'] == 10)),
        sum(x['nps_100'] <= 6),
        sum((x['nps_100'] == 7) | (x['nps_100'] == 8))
    ))

    # Calculate promoters_weight and detractors_weight
    aggregated_features_data['promoters_weight'] = grouped_data.apply(lambda x: (x[x['nps_100'] > 8]['monthly_weight'].sum()) if (x['nps_100'] > 8).any() else 0)
    aggregated_features_data['detractors_weight'] = grouped_data.apply(lambda x: (x[x['nps_100'] <= 6]['monthly_weight'].sum()) if (x['nps_100'] <= 6).any() else 0)




    # Calculate monthly_weight within each group
    aggregated_features_data['monthly_weight'] = grouped_data.apply(lambda x: x['monthly_weight'].sum())

    # Calculate NPS_weighted using the same logic as your SQL query
    aggregated_features_data['NPS_weighted'] = (aggregated_features_data['promoters_weight'] - aggregated_features_data['detractors_weight']) * 100 / aggregated_features_data['monthly_weight']

    for col in touchpoints + non_touchpoints:
        aggregated_features_data[f'{col}_sum'] = grouped_data[col].sum()
        aggregated_features_data[f'{col}_std'] = grouped_data[col].std()
        aggregated_features_data[f'{col}_mean'] = grouped_data[col].mean()
        aggregated_features_data[f'{col}_not_nulls'] = grouped_data[col].apply(lambda x: x.notnull().sum())
        if col in touchpoints:
            aggregated_features_data[f'{col}_satisfaction'] = grouped_data[col].apply(lambda x: x[x >= 8].count() / x.count()*100)
        else:
            aggregated_features_data[f'{col}_max'] = grouped_data[col].max()
            aggregated_features_data[f'{col}_min'] = grouped_data[col].min()
    
    # Reset the index of aggregated_features_data
    aggregated_features_data.reset_index(inplace=True)

    # Split the 'cross_kpis' column back into individual columns and concatenate them to aggregated_features_data
    split_columns = aggregated_features_data['cross_kpis'].str.split('_', expand=True)
    split_columns.columns = columns_to_cross_kpis
    aggregated_features_data = pd.concat([split_columns, aggregated_features_data], axis=1)
    
    # Drop the temporary 'cross_kpis' column
    aggregated_features_data.drop(columns=['cross_kpis'], inplace=True)
    
    return aggregated_features_data


In [25]:
df=calculate_aggregated_features_with_cross_kpis(df_nps_tkt, touchpoints, columns_to_cross_kpis, time_frequency='M', calculate_satisfaction=True)

In [26]:
df[df['date_flight_local']>'2023-03-07']

,cabin_in_surveyed_flight,haul,date_flight_local,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction
50,Business,LH,2023-03-31,42.154,114.582,36.289,185.731,42.154,4322.000,2.900,7.916,546,73.810,5181.000,2.633,8.185,633,77.093,5339.000,2.109,8.502,628,80.414,4196.000,2.338,8.148,515,73.592,5212.000,2.444,8.247,632,76.741,5457.000,2.164,8.487,643,78.072,5358.000,1.993,8.628,621,80.998,2303.000,3.322,6.362,362,44.751,4327.000,2.386,8.088,535,74.579,4972.000,2.630,7.818,636,67.925,5559.000,2.130,8.700,639,82.942,5401.000,2.261,8.413,642,78.660,2369.000,2.825,7.870,301,71.429,5555.000,2.564,8.546,650,80.923,3213.000,2.041,8.134,395,71.392,304.000,3.037,2.789,109,9.174,1183.000,3.341,6.878,172,56.395
51,Business,LH,2023-04-30,45.176,151.797,41.816,243.448,45.176,5913.000,2.473,8.111,729,74.211,6983.000,2.120,8.537,818,81.418,6984.000,1.933,8.654,807,84.139,5578.000,2.301,8.325,670,77.463,6955.000,2.252,8.451,823,79.951,7192.000,1.945,8.603,836,81.100,7252.000,1.694,8.801,824,85.680,2498.000,3.335,6.276,398,44.221,5726.000,2.200,8.227,696,75.287,6485.000,2.631,7.785,833,68.307,7345.000,1.994,8.796,835,84.311,7180.000,2.051,8.599,835,82.395,2794.000,2.831,7.938,352,72.727,7638.000,1.979,8.986,850,87.294,4160.000,2.204,8.173,509,75.639,412.000,3.512,3.583,115,19.130,1399.000,3.123,6.926,202,55.446
52,Business,LH,2023-05-31,50.318,166.056,40.656,249.216,50.318,6404.000,2.527,8.200,781,77.081,7777.000,2.192,8.622,902,82.594,7933.000,1.758,8.795,902,85.144,4284.000,2.223,8.335,514,76.654,7859.000,2.079,8.655,908,81.498,8053.000,2.047,8.631,933,82.315,8086.000,1.699,8.808,918,84.967,3080.000,3.426,6.197,497,47.082,6397.000,2.178,8.233,777,75.161,7578.000,2.342,8.148,930,

### Add data from the dasboard's last month manually

### Merge dfs

In [27]:
# Convert the 'date_flight_local' column in result_df to separate 'year' and 'month' columns
df['year'] = df['date_flight_local'].dt.year
df['month'] = df['date_flight_local'].dt.month

# Perform the merge based on the specified columns
merged_df = df.merge(df_kpis, 
                            left_on=['year', 'month', 'cabin_in_surveyed_flight', 'haul'], 
                            right_on=['year', 'month', 'cabin', 'haul'], 
                            how='inner')

# Drop the redundant columns from the merged dataframe
merged_df.drop(columns=['cabin_in_surveyed_flight'], inplace=True)

# Reorder the columns to place 'year', 'month', 'cabin', and 'haul' at the beginning
cols_to_move = ['date_flight_local','year', 'month', 'cabin', 'haul']
merged_df = merged_df[cols_to_move + [col for col in merged_df.columns if col not in cols_to_move]]
merged_df.head()

,date_flight_local,year,month,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
0,2019-01-31,2019,1,Business,LH,49.086,121.666,25.560,195.790,49.086,2562.000,2.043,8.428,304,78.618,3283.000,1.788,8.662,379,84.960,3086.000,1.679,8.525,362,81.215,2443.000,2.066,7.906,309,68.932,3119.000,1.881,8.144,383,71.279,3188.000,1.813,8.324,383,76.240,3221.000,1.780,8.476,380,79.737,1110.000,3.395,5.550,200,37.000,2471.000,1.810,7.971,310,67.742,2986.000,2.070,7.837,381,66.142,3266.000,1.854,8.527,383,80.157,3089.000,1.835,8.259,374,76.203,371.000,1.569,8.244,45,71.111,3426.000,1.928,8.945,383,87.728,2644.000,1.933,7.869,336,66.071,164.000,3.288,4.100,40,17.500,2212.000,2.485,7.135,310,53.548,76.168,80.513,87.692
1,2019-02-28,2019,2,Business,LH,46.939,113.669,26.158,186.436,46.939,2654.000,2.002,8.425,315,80.000,3328.000,1.837,8.644,385,85.455,3197.000,1.747,8.480,377,79.310,2635.000,2.134,7.961,331,68.580,3177.000,1.946,8.125,391,72.379,3256.000,1.834,8.306,392,77.551,3217.000,1.802,8.444,381,79.003,1135.000,3.205,5.821,195,39.487,2632.000,1.942,8.074,326,74.847,3039.000,2.079,7.853,387,67.442,3284.000,2.135,8.378,392,78.061,3137.000,1.794,8.212,382,77.487,250.000,1.688,8.333,30,70.000,3479.000,1.988,8.875,392,85.204,2664.000,2.029,7.882,338,67.751,177.000,3.411,4.658,38,21.053,2130.000,2.667,7.030,303,55.776,78.736,80.543,89.501
2,2019-03-31,2019,3,Business,LH,46.512,135.858,30.774,225.929,46.512,1949.000,2.263,8.087,241,73.859,2488.000,1.998,8.405,296,80.743,2369.000,2.116,8.197,289,75.779,1859.000,2.280,7.682,242,66.529,2430.000,2.084,8.073,301,73.754,2524.000,1.698,8.385,301,79.734,2517.000,1.597,8.446,298,80.201,85

In [28]:
merged_df.describe()

,date_flight_local,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
count,290,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,280.000,284.000,290.000,284.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,283.000,286.000,290.000,286.000,290.000,284.000,286.000,290.000,286.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,289.000,290.000,290.000,290.000,290.000,283.000,285.000,290.000,285.000,290.000,290.000,290.000,290.000,290.000,290.000,289.000,289.000,290.000,289.000,290.000,285.000,290.000,290.000,290.000,290.000,289.000,290.000,290.000,290.000,290.000,290.000,290.000
mean,2021-06-18 19:46:45.517241344,31.636,966.918,416.410,1808.048,31.636,12733.866,2.734,7.543,1695.241,65.974,15234.969,2.417,7.972,1913.828,72.872,15737.324,2.061,8.232,1903.031,76.061,1682.338,2.313,7.833,212.152,69.468,15253.038,2.312,7.872,1934.500,70.047,15393.779,2.161,8.024,1940.124,72.069,15749.655,2.001,8.327,1898.152,76.654,2060.586,3.448,4.918,377.779,30.372,4105.224,2.531,7.387,528.979,62.989,8724.172,2.730,6.740,1291.172,50.861,15831.417,2.255,8.182,1908.648,75.172,15289.228,2.198,7.966,1910.093,71.363,4246.052,2.767,7.372,576.362,63.986,16195.000,2.392,8.524,1963.528,81.929,6414.476,2.250,7.431,873.590,58.431,749.648,3.211,3.154,255.734,14.445,5525.679,3.154,6.170,912.145,44.352,71.884,85.965,87.675
min,2019-01-31 00:00:00,-30.000,0.655,0.505,1.310,-30.000,13.000,1.904,5.167,2.000,43.636,15.000,1

In [29]:
merged_df['date_flight_local'].max()

Timestamp('2023-10-31 00:00:00')

## 2) Change of survey adjustment.

Since the adjustment is done separatelly in the global NPS and in the Cabin/haul NPS we need to have a separate dataframe for the NPS.  

In [30]:
# Sample adjustments DataFrame
adjustments_data = {'haul': ['LH', 'LH', 'LH', 'SH', 'SH', 'SH'],
                    'cabin': ['Economy', 'Premium Economy' , 'Business', 'Economy', 'Premium Economy', 'Business'],
                    'adjustment_2019': [-0.55475719, -0.55475719, -6.86337594, -5.25927601, -5.25927601, -5.64891133],
                    'adjustment_2022': [1.41878561, 1.41878561, 0.42510779, -3.71320153, -3.71320153, -4.10337349]}

adjustments_df = pd.DataFrame(adjustments_data)
df=merged_df.copy()

for index, row in adjustments_df.iterrows():
    condition = ((df['haul'] == row['haul']) | (df['haul'] == 'MH')) & (df['cabin'] == row['cabin'])
    
    mask = (df['date_flight_local'].dt.year == 2019) & (df['date_flight_local'].dt.month != 12)
    df.loc[condition & mask, 'NPS'] += row['adjustment_2019']
    df.loc[condition & mask, 'NPS_weighted'] += row['adjustment_2019']
    
    mask = (df['date_flight_local'].dt.year == 2019) & (df['date_flight_local'].dt.month == 12)
    df.loc[condition & mask, 'NPS'] += row['adjustment_2022']
    df.loc[condition & mask, 'NPS_weighted'] += row['adjustment_2022']
    
    mask = (df['date_flight_local'].dt.year == 2022)
    df.loc[condition & mask, 'NPS'] += row['adjustment_2022']
    df.loc[condition & mask, 'NPS_weighted'] += row['adjustment_2022']
    
df[df['date_flight_local'] == '2022-12-31'].head()


,date_flight_local,year,month,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
47,2022-12-31,2022,12,Business,LH,30.728,142.978,64.634,258.535,30.728,849.000,2.564,7.513,113,62.832,1084.000,2.205,8.275,131,79.389,1030.000,2.080,7.984,129,65.891,843.000,2.484,7.664,110,66.364,1057.000,2.025,8.008,132,73.485,1099.000,1.822,8.326,132,76.515,1100.000,1.909,8.594,128,79.688,439.000,3.212,5.853,75,36.000,723.000,2.285,7.859,92,67.391,956.000,2.694,7.411,129,62.791,1098.000,2.282,8.382,131,77.863,1068.000,2.017,8.153,131,76.336,242.000,2.944,6.914,35,57.143,1085.000,2.607,8.220,132,76.515,810.000,2.274,7.364,110,60.909,61.000,3.187,3.050,20,15.000,713.000,3.016,6.482,110,47.273,85.048,73.067,75.426
105,2022-12-31,2022,12,Business,SH,37.053,366.288,118.685,601.612,37.053,1854.000,2.735,7.537,246,66.260,2325.000,2.197,8.304,280,80.357,2236.000,2.458,8.014,279,72.760,1445.000,2.450,8.028,180,72.778,2266.000,2.348,7.787,291,67.010,2365.000,2.231,8.099,292,76.370,2359.000,2.055,8.277,285,78.246,513.000,2.988,5.897,87,29.885,70.000,4.296,6.364,11,54.545,2146.000,2.390,7.747,277,66.426,2551.000,1.901,8.766,291,86.942,2302.000,2.183,8.106,284,70.775,766.000,3.177,7.437,103,66.019,2403.000,2.743,8.173,294,77.211,1459.000,1.971,7.599,192,56.250,129.000,3.127,3.146,41,14.634,1238.000,3.072,6.516,190,47.895,62.256,85.953,88.466
163,2022-12-31,2022,12,Economy,LH,17.597,1186.707,774.557,2547.512,17.597,4626.000,2.960,7.095,652,58.742,5242.000,2.798,7.394,709,62.764,5469.000,2.365,7.835,698,67.908,468.000,1.750,8.211,57,71.930,5336.000,2.417,7.515,710,61.408,5258.000,2.643,7.406,710,61.549,5844.000,2.277,8.266,707,76.379,1235.00

Lets check the NPS curve now against the dashboard:

In [31]:
shp_bi = pd.read_excel('inputs/SH Premium.xlsx')
shp_bi

,Quarter,Month_Name,2019,2022,2023
0,Q1,Jan,38.240,44.246,35.708
1,Q1,Feb,46.748,34.647,48.818
2,Q1,Mar,40.184,38.806,35.570
3,Q2,Apr,36.272,32.042,45.796
4,Q2,May,46.937,27.681,44.219
5,Q2,Jun,18.644,25.831,40.074
6,Q3,Jul,33.755,28.475,41.470
7,Q3,Aug,35.874,36.682,48.652
8,Q3,Sep,33.372,28.024,NaN
9,Q4,Oct,43.783,44.776,NaN


In [32]:
data=df[(df['date_flight_local'].dt.year.isin([2019, 2022, 2023])) & (df['cabin'] == 'Business') & (df['haul'] == 'SH')][['NPS', 'NPS_weighted', 'date_flight_local']]


In [33]:
# Pivot the dataframe
pivot_df = data.pivot_table(index=data['date_flight_local'].dt.month, columns=data['date_flight_local'].dt.year, values=['NPS', 'NPS_weighted'], aggfunc='mean')

# Rename the columns
pivot_df.columns = [f"{year}" for _, year in pivot_df.columns]

# Reset the index
pivot_df.reset_index(inplace=True)


pivot_df.rename(columns={'date_flight_local': 'Month_Name'}, inplace=True)

In [34]:
pivot_df

,Month_Name,2019,2022,2023,2019,2022,2023
0,1,38.240,44.246,35.708,38.240,44.246,35.708
1,2,46.748,34.647,48.818,46.748,34.647,48.818
2,3,40.184,38.806,40.043,40.184,38.806,40.043
3,4,36.272,31.208,45.796,36.272,32.042,45.796
4,5,46.937,27.681,44.219,46.937,27.681,44.219
5,6,18.644,25.831,40.074,18.644,25.831,40.074
6,7,33.755,28.475,41.470,33.755,28.475,41.470
7,8,35.874,36.682,49.729,35.874,36.682,49.729
8,9,33.372,28.024,38.735,33.372,28.024,38.735
9,10,43.783,44.776,43.295,43.783,44.776,43.295


#### Touchpoints' satisfaction  adjustment for the change of survey

In [35]:
df['date_flight_local']=pd.to_datetime(df['date_flight_local'])

In [36]:
import pandas as pd
from io import StringIO

#Flight Booking
data = """
Touchpoint
IB Plus loyalty program
Boarding
Ease of contact by phone
IFE
Connections experience
In flight food and beverage
Airport security
Wi-Fi
Lounge
Arrivals experience
Check-in
Journey preparation support
Pilot's announcements
Aircraft interior
Cabin Crew
Punctuality
Response provided to the issue
"""
#5.0
data2 = """
Global adjustment
12.7
8.2
7.0
6.9
5.7
5.6
5.2
5.2
5.1
4.5
3.3
3.2
2.5
2.4
1.1
-0.2
-3.1
"""
#3.6
data3 = """
SH_Business
16.1
4.0
0.8
1.0
3.3
4.0
1.5
10.2
1.5
3.1
-1.2
4.8
-0.7
0.5
-3.0
-2.5
-2.6
"""
#7.5
data4 = """
LH_Business
11.3
5.5
11.4
5.0
12.6
3.3
14.0
2.3
8.2
6.2
0.8
7.9
1.2
2.1
3.6
1.2
3.4
"""
#4.6
data5 = """
SH_Economy
12.7
8.1
5.2
3.4
4.2
5.6
3.8
6.5
6.7
3.8
3.2
3.0
2.3
2.7
1.5
-0.8
-3.8
"""
#4.6
data5 = """
SH_Premium Economy
12.7
8.1
5.2
3.4
4.2
5.6
3.8
6.5
6.7
3.8
3.2
3.0
2.3
2.7
1.5
-0.8
-3.8
"""
#7.4
data6 = """
LH_Economy
11.2
10.5
10.1
7.4
10.0
8.6
9.6
1.8
9.8
7.5
7.0
3.6
3.5
4.4
3.9
1.7
-1.1
"""
#7.4
data7 = """
LH_Premium Economy
11.2
10.5
10.1
7.4
10.0
8.6
9.6
1.8
9.8
7.5
7.0
3.6
3.5
4.4
3.9
1.7
-1.1
"""

# Create StringIO objects to simulate file-like input
data_stream = StringIO(data)
data2_stream = StringIO(data2)
data3_stream = StringIO(data3)
data4_stream = StringIO(data4)
data5_stream = StringIO(data5)
data6_stream = StringIO(data6)
data7_stream = StringIO(data7)

# Create DataFrames
ta_df = pd.DataFrame({'Touchpoint': data_stream.getvalue().split('\n')[1:]})
ta_df['Global adjustment'] = data2_stream.getvalue().split('\n')[1:]
ta_df['SH_Business'] = data3_stream.getvalue().split('\n')[1:]
ta_df['LH_Business'] = data4_stream.getvalue().split('\n')[1:]
ta_df['SH_Economy'] = data5_stream.getvalue().split('\n')[1:]
ta_df['LH_Economy'] = data6_stream.getvalue().split('\n')[1:]
ta_df['LH_Premium Economy'] = data7_stream.getvalue().split('\n')[1:]

# Display the DataFrame
ta_df=ta_df.drop(0)
ta_df=ta_df.drop(18)
ta_df


,Touchpoint,Global adjustment,SH_Business,LH_Business,SH_Economy,LH_Economy,LH_Premium Economy
1,IB Plus loyalty program,12.7,16.1,11.3,12.7,11.2,11.2
2,Boarding,8.2,4.0,5.5,8.1,10.5,10.5
3,Ease of contact by phone,7.0,0.8,11.4,5.2,10.1,10.1
4,IFE,6.9,1.0,5.0,3.4,7.4,7.4
5,Connections experience,5.7,3.3,12.6,4.2,10.0,10.0
6,In flight food and beverage,5.6,4.0,3.3,5.6,8.6,8.6
7,Airport security,5.2,1.5,14.0,3.8,9.6,9.6
8,Wi-Fi,5.2,10.2,2.3,6.5,1.8,1.8
9,Lounge,5.1,1.5,8.2,6.7,9.8,9.8
10,Arrivals experience,4.5,3.1,6.2,3.8,7.5,7.5


In [37]:
satisfaction_mapping = {
    'IB Plus loyalty program': 'loy_200_loyalty_programme_satisfaction',
    'Boarding': 'pfl_500_boarding_satisfaction',
    'Ease of contact by phone': 'img_310_ease_contact_phone_satisfaction',
    'IFE': 'ifl_500_ife_satisfaction',
    'Connections experience': 'con_100_connections_satisfaction',
    'In flight food and beverage': 'ifl_400_food_drink_satisfaction',
    'Airport security': 'pfl_200_security_satisfaction',
    'Wi-Fi': 'ifl_600_wifi_satisfaction',
    'Lounge': 'pfl_300_lounge_satisfaction',
    #'Flight Booking': 'bkg_100_booking_satisfaction',
    'Arrivals experience': 'arr_100_arrivals_satisfaction',
    'Check-in': 'pfl_100_checkin_satisfaction',
    'Journey preparation support': 'bkg_200_journey_preparation_satisfaction',
    "Pilot's announcements": 'ifl_200_flight_crew_annoucements_satisfaction',
    'Aircraft interior': 'ifl_300_cabin_satisfaction',
    'Cabin Crew': 'ifl_100_cabin_crew_satisfaction',
    'Punctuality': 'pun_100_punctuality_satisfaction',
    'Response provided to the issue': 'inm_400_issues_response_satisfaction'
}

# Create a Series to use for mapping
satisfaction_series = pd.Series(satisfaction_mapping)

# Replace touchpoints with their satisfaction equivalents in the original DataFrame
ta_df['Touchpoint'] = ta_df['Touchpoint'].map(satisfaction_series)


#Flip the dataframe to match the NPS one:
flipped_df = ta_df.transpose()
flipped_df.columns = flipped_df.iloc[0]
ta_df = flipped_df.drop(flipped_df.index[0])
ta_df


Touchpoint,loy_200_loyalty_programme_satisfaction,pfl_500_boarding_satisfaction,img_310_ease_contact_phone_satisfaction,ifl_500_ife_satisfaction,con_100_connections_satisfaction,ifl_400_food_drink_satisfaction,pfl_200_security_satisfaction,ifl_600_wifi_satisfaction,pfl_300_lounge_satisfaction,arr_100_arrivals_satisfaction,pfl_100_checkin_satisfaction,bkg_200_journey_preparation_satisfaction,ifl_200_flight_crew_annoucements_satisfaction,ifl_300_cabin_satisfaction,ifl_100_cabin_crew_satisfaction,pun_100_punctuality_satisfaction,inm_400_issues_response_satisfaction
Global adjustment,12.7,8.2,7.0,6.9,5.7,5.6,5.2,5.2,5.1,4.5,3.3,3.2,2.5,2.4,1.1,-0.2,-3.1
SH_Business,16.1,4.0,0.8,1.0,3.3,4.0,1.5,10.2,1.5,3.1,-1.2,4.8,-0.7,0.5,-3.0,-2.5,-2.6
LH_Business,11.3,5.5,11.4,5.0,12.6,3.3,14.0,2.3,8.2,6.2,0.8,7.9,1.2,2.1,3.6,1.2,3.4
SH_Economy,12.7,8.1,5.2,3.4,4.2,5.6,3.8,6.5,6.7,3.8,3.2,3.0,2.3,2.7,1.5,-0.8,-3.8
LH_Economy,11.2,10.5,10.1,7.4,10.0,8.6,9.6,1.8,9.8,7.5,7.0,3.6,3.5,4.4,3.9,1.7,-1.1
LH_Premium Economy,11.2,10.5,10.1,7.4,10.0,8.6,9.6,1.8,9.8,7.5,7.0,3.6,3.5,4.4,3.9,1.7,-1.1


In [38]:
df.describe()

,date_flight_local,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
count,290,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,280.000,284.000,290.000,284.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,283.000,286.000,290.000,286.000,290.000,284.000,286.000,290.000,286.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,290.000,289.000,290.000,290.000,290.000,290.000,283.000,285.000,290.000,285.000,290.000,290.000,290.000,290.000,290.000,290.000,289.000,289.000,290.000,289.000,290.000,285.000,290.000,290.000,290.000,290.000,289.000,290.000,290.000,290.000,290.000,290.000,290.000
mean,2021-06-18 19:46:45.517241344,30.715,966.918,416.410,1808.048,30.716,12733.866,2.734,7.543,1695.241,65.974,15234.969,2.417,7.972,1913.828,72.872,15737.324,2.061,8.232,1903.031,76.061,1682.338,2.313,7.833,212.152,69.468,15253.038,2.312,7.872,1934.500,70.047,15393.779,2.161,8.024,1940.124,72.069,15749.655,2.001,8.327,1898.152,76.654,2060.586,3.448,4.918,377.779,30.372,4105.224,2.531,7.387,528.979,62.989,8724.172,2.730,6.740,1291.172,50.861,15831.417,2.255,8.182,1908.648,75.172,15289.228,2.198,7.966,1910.093,71.363,4246.052,2.767,7.372,576.362,63.986,16195.000,2.392,8.524,1963.528,81.929,6414.476,2.250,7.431,873.590,58.431,749.648,3.211,3.154,255.734,14.445,5525.679,3.154,6.170,912.145,44.352,71.884,85.965,87.675
min,2019-01-31 00:00:00,-30.000,0.655,0.505,1.310,-30.000,13.000,1.904,5.167,2.000,43.636,15.000,1

In [39]:
original_df=df.copy()
adjustment_df=ta_df.drop("Global adjustment")

# Convert the values in the adjustment_df to numeric
adjustment_df = adjustment_df.apply(pd.to_numeric, errors='coerce')

# Filter out the rows where the year is 2023
adjusted_years = original_df['date_flight_local'].dt.year != 2023
adjusted_df = original_df[adjusted_years].copy()

# Add the adjustments to the corresponding touchpoint columns
for index, row in adjustment_df.iterrows():
    haul, cabin = index.split('_')
    mask = (adjusted_df['haul'] == haul) & (adjusted_df['cabin'] == cabin)
    for touchpoint_col in adjustment_df.columns:
        adjusted_df[touchpoint_col].fillna(0, inplace=True) 
        adjusted_df.loc[mask, touchpoint_col] += row[touchpoint_col]
        
# Concatenate adjusted_df with values from 2023
adjusted_df = pd.concat([adjusted_df, original_df[~adjusted_years]])




In [40]:
adjusted_df = adjusted_df[~((adjusted_df['cabin'] == 'Premium Economy') & (adjusted_df['haul'] == 'SH'))]


In [41]:
adjusted_df

,date_flight_local,year,month,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
0,2019-01-31,2019,1,Business,LH,42.223,121.666,25.560,195.790,42.223,2562.000,2.043,8.428,304,86.518,3283.000,1.788,8.662,379,85.760,3086.000,1.679,8.525,362,95.215,2443.000,2.066,7.906,309,77.132,3119.000,1.881,8.144,383,76.779,3188.000,1.813,8.324,383,78.340,3221.000,1.780,8.476,380,80.937,1110.000,3.395,5.550,200,39.300,2471.000,1.810,7.971,310,72.742,2986.000,2.070,7.837,381,69.442,3266.000,1.854,8.527,383,83.757,3089.000,1.835,8.259,374,82.403,371.000,1.569,8.244,45,83.711,3426.000,1.928,8.945,383,88.928,2644.000,1.933,7.869,336,77.371,164.000,3.288,4.100,40,20.900,2212.000,2.485,7.135,310,64.948,76.168,80.513,87.692
1,2019-02-28,2019,2,Business,LH,40.075,113.669,26.158,186.436,40.075,2654.000,2.002,8.425,315,87.900,3328.000,1.837,8.644,385,86.255,3197.000,1.747,8.480,377,93.310,2635.000,2.134,7.961,331,76.780,3177.000,1.946,8.125,391,77.879,3256.000,1.834,8.306,392,79.651,3217.000,1.802,8.444,381,80.203,1135.000,3.205,5.821,195,41.787,2632.000,1.942,8.074,326,79.847,3039.000,2.079,7.853,387,70.742,3284.000,2.135,8.378,392,81.661,3137.000,1.794,8.212,382,83.687,250.000,1.688,8.333,30,82.600,3479.000,1.988,8.875,392,86.404,2664.000,2.029,7.882,338,79.051,177.000,3.411,4.658,38,24.453,2130.000,2.667,7.030,303,67.176,78.736,80.543,89.501
2,2019-03-31,2019,3,Business,LH,39.648,135.858,30.774,225.929,39.648,1949.000,2.263,8.087,241,81.759,2488.000,1.998,8.405,296,81.543,2369.000,2.116,8.197,289,89.779,1859.000,2.280,7.682,242,74.729,2430.000,2.084,8.073,301,79.254,2524.000,1.698,8.385,301,81.834,2517.000,1.597,8.446,298,81.401,85

In [42]:
# Drop duplicates based on the combination of three columns: 'date_flight_local', 'cabin', and 'haul'
adjusted_df.drop_duplicates(subset=['date_flight_local', 'cabin', 'haul'], inplace=True)


In [43]:
adjusted_df.reset_index()

,index,date_flight_local,year,month,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
0,0,2019-01-31,2019,1,Business,LH,42.223,121.666,25.560,195.790,42.223,2562.000,2.043,8.428,304,86.518,3283.000,1.788,8.662,379,85.760,3086.000,1.679,8.525,362,95.215,2443.000,2.066,7.906,309,77.132,3119.000,1.881,8.144,383,76.779,3188.000,1.813,8.324,383,78.340,3221.000,1.780,8.476,380,80.937,1110.000,3.395,5.550,200,39.300,2471.000,1.810,7.971,310,72.742,2986.000,2.070,7.837,381,69.442,3266.000,1.854,8.527,383,83.757,3089.000,1.835,8.259,374,82.403,371.000,1.569,8.244,45,83.711,3426.000,1.928,8.945,383,88.928,2644.000,1.933,7.869,336,77.371,164.000,3.288,4.100,40,20.900,2212.000,2.485,7.135,310,64.948,76.168,80.513,87.692
1,1,2019-02-28,2019,2,Business,LH,40.075,113.669,26.158,186.436,40.075,2654.000,2.002,8.425,315,87.900,3328.000,1.837,8.644,385,86.255,3197.000,1.747,8.480,377,93.310,2635.000,2.134,7.961,331,76.780,3177.000,1.946,8.125,391,77.879,3256.000,1.834,8.306,392,79.651,3217.000,1.802,8.444,381,80.203,1135.000,3.205,5.821,195,41.787,2632.000,1.942,8.074,326,79.847,3039.000,2.079,7.853,387,70.742,3284.000,2.135,8.378,392,81.661,3137.000,1.794,8.212,382,83.687,250.000,1.688,8.333,30,82.600,3479.000,1.988,8.875,392,86.404,2664.000,2.029,7.882,338,79.051,177.000,3.411,4.658,38,24.453,2130.000,2.667,7.030,303,67.176,78.736,80.543,89.501
2,2,2019-03-31,2019,3,Business,LH,39.648,135.858,30.774,225.929,39.648,1949.000,2.263,8.087,241,81.759,2488.000,1.998,8.405,296,81.543,2369.000,2.116,8.197,289,89.779,1859.000,2.280,7.682,242,74.729,2430.000,2.084,8.073,301,79.254,2524.000,1.698,8.385,301,81.834,2517.000,1.597,8.446,2

In [44]:
adjusted_df.to_csv(outputs_path, index=False)